In [2]:
from sklearn.model_selection import train_test_split

from pstree.cluster_gp_sklearn import PSTreeRegressor, GPRegressor
from pstree.datasets.data_loader import load_concrete_strength
from pstree.datasets.synthetic_datasets import load_synthetic1
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, root_mean_squared_error as rmse

from matplotlib import pyplot as plt

In [3]:
X,y = load_concrete_strength()

In [ ]:
X,y = load_diabetes()
# X, y, _, _ = load_synthetic1()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

r = PSTreeRegressor(regr_class=GPRegressor, 
                    tree_class=DecisionTreeRegressor,
                    height_limit=6, 
                    n_pop=25,  # 25
                    n_gen=50,  # 50 
                    basic_primitive=True,
                    size_objective=True,
                    max_leaf_nodes=4,
                    constant_range=2,  # 2 
                    random_seed=0, 
                    random_state=0,
                    normalize=True,
                    verbose=True,
)

r.fit(X_train, y_train)
print(r2_score(y_test, r.predict(X_test)))
print(rmse(y_test, r.predict(X_test)))

plt.figure(figsize=(10, 6))
plt.scatter(y_test, r.predict(X_test), alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title('PSTreeRegressor Predictions vs True Values')
plt.show()

In [ ]:
r.regr.

array([[ 1.49836523,  1.06136988,  0.21990201, ...,  0.71103773,
         0.54756035, -0.06144896],
       [-0.22885822,  1.06136988, -0.41936607, ...,  1.4842858 ,
        -0.01977619,  0.36723647],
       [ 0.08518241, -0.94217861,  1.01898711, ..., -0.06221033,
         0.33126035, -0.31866022],
       ...,
       [ 0.63475351, -0.94217861, -0.46502808, ..., -0.83545839,
        -0.25377593, -0.06144896],
       [-0.30736838, -0.94217861, -0.53352109, ..., -0.06221033,
        -0.83071152, -0.83308273],
       [-2.03459183, -0.94217861,  0.56236706, ..., -0.83545839,
        -0.13319219, -0.06144896]], shape=(353, 10))